# Rupaul's Drag Race Machine

In [2]:
import pandas as pd
from scipy.stats import rankdata, kendalltau

theData = pd.read_csv("dragrace.csv")
theData= theData.fillna(0)
#theData.iloc[1:10,:]

In [3]:
queens = theData.groupby('Name').max()
queens = queens.reset_index(drop=False)
#queens.iloc[1:10,:]

In [3]:
#queens.loc[queens.Place==1,:]

In [4]:
Xtrain = queens.loc[queens.Season<7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
Xtest = queens.loc[queens.Season==7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
ytrain = queens.loc[queens.Season<7,'Place']
ytest = queens.loc[queens.Season==7,'Place']
season8X = queens.loc[queens.Season==8,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
season8y = queens.loc[queens.Season==8,'Place']
season7 = queens.loc[queens.Season==7,['Name','Place']]
season7.columns = ['Name','Actual']
season7 = season7.reset_index(drop=True)
season8 = queens.loc[queens.Season==8,['Name','Place']]
season8.columns = ['Name','Actual']
season8 = season8.reset_index(drop=True)

In [5]:
def check_ranks(real,predicted):
    mymean = real.mean()
    x2 = real-mymean
    x2 = x2*x2
    sum1 = x2.sum()
    x2 = real-predicted
    x2 = x2*x2
    sum2 = x2.sum()
    r2 = 1 - (sum2/sum1)
    return r2

# Support Vector Machine Classifier

In [6]:
from sklearn.svm import SVC
model = SVC()
model.fit(Xtrain,ytrain)

yfit = model.predict(Xtest)
yfitpd = pd.DataFrame(yfit)
yfitpd['Name'] = season7.Name
yfitpd.columns = ['Predicted','Name']
yfitpd = yfitpd.loc[:,['Name','Predicted']]
yfitpd['Predicted'] = rankdata(yfitpd.Predicted,method='min')
svc7 = pd.merge(season7,yfitpd).sort_values('Actual')
svc7

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,5
9,Pearl,3,9
5,Kennedy Davenport,4,5
4,Katya,5,5
12,Trixie Mattel,6,9
7,Miss Fame,7,3
1,Jaidynn Diore Fierce,8,12
6,Max,9,9
3,Kandy Ho,10,4


In [7]:
kendalltau(svc7.Actual,svc7.Predicted)

KendalltauResult(correlation=0.22130457664129397, pvalue=0.27024792506861006)

## Season 8 Predictions

In [8]:
presentFit = model.predict(season8X)
presentFitPD = pd.DataFrame(presentFit)
presentFitPD['Name'] = season8['Name']
presentFitPD.columns = ['Predicted','Name']
presentFitPD['Predicted'] = rankdata(presentFitPD.Predicted,method='min')
svc8 = pd.merge(season8,presentFitPD).sort_values('Predicted')
svc8

,Name,Actual,Predicted
0,Acid Betty,0,1
4,Dax ExclamationPoint,11,2
5,Derrick Barry,0,2
9,Naysha Lopez,9,2
10,Robbie Turner,0,2
1,Bob the Drag Queen,0,6
2,Chi Chi DeVayne,0,6
3,Cynthia Lee Fontaine,10,6
8,Naomi Smalls,0,6
11,Thorgy Thor,0,6


# Gaussian Naive Bayes

In [9]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(Xtrain,ytrain)
nbTest = model.predict(Xtest)
nbTestDB = pd.DataFrame(nbTest)
nbTestDB['Name'] = season7.Name
nbTestDB.columns = ['Predicted','Name']
nbTestDB = nbTestDB.loc[:,['Name','Predicted']]
nbTestDB['Predicted'] = rankdata(nbTestDB.Predicted,method='min')

nb7 = pd.merge(season7,nbTestDB).sort_values('Actual')
nb7

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,6
9,Pearl,3,8
5,Kennedy Davenport,4,1
4,Katya,5,1
12,Trixie Mattel,6,8
7,Miss Fame,7,4
1,Jaidynn Diore Fierce,8,6
6,Max,9,8
3,Kandy Ho,10,4


In [10]:
kendalltau(nb7.Actual,nb7.Predicted)

KendalltauResult(correlation=0.48108780653777095, pvalue=0.016544547824374332)

## Season 8 Predictions

In [11]:
nb8 = model.predict(season8X)
nb8DB = pd.DataFrame(nb8)
nb8DB['Name'] = season8.Name
nb8DB.columns = ['Predicted','Name']
nb8DB['Predicted'] = rankdata(nb8DB.Predicted,method='min')
nb8 = pd.merge(season8,nb8DB).sort_values('Predicted')
nb8

,Name,Actual,Predicted
0,Acid Betty,0,1
6,Kim Chi,0,1
5,Derrick Barry,0,3
3,Cynthia Lee Fontaine,10,4
9,Naysha Lopez,9,4
1,Bob the Drag Queen,0,6
2,Chi Chi DeVayne,0,6
8,Naomi Smalls,0,6
10,Robbie Turner,0,6
11,Thorgy Thor,0,6


# Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(Xtrain,ytrain)
yfit = model.predict(Xtest)

yfitpd = pd.DataFrame(yfit)
yfitpd['Name'] = season7.Name
yfitpd.columns = ['Predicted','Name']
yfitpd = yfitpd.loc[:,['Name','Predicted']]
yfitpd['Predicted'] = rankdata(yfitpd.Predicted,method='min')

## Season 7 Predictions

In [13]:
rfClass = pd.merge(season7,yfitpd).sort_values('Actual')
rfClass

,Name,Actual,Predicted
13,Voilet Chachki,1,1
0,Ginger Minj,2,6
9,Pearl,3,3
5,Kennedy Davenport,4,4
4,Katya,5,4
12,Trixie Mattel,6,6
7,Miss Fame,7,10
1,Jaidynn Diore Fierce,8,6
6,Max,9,1
3,Kandy Ho,10,6


In [14]:
kendalltau(rfClass.Actual,rfClass.Predicted)

KendalltauResult(correlation=0.64061851133006142, pvalue=0.0014157354382150771)

## Season 8 Predictions

In [15]:
rf8Fit = model.predict(season8X)
rf8FitPD = pd.DataFrame(rf8Fit)
rf8FitPD['Name'] = season8.Name
rf8FitPD.columns = ['Predicted','Name']
rf8FitPD = rf8FitPD.loc[:,['Name','Predicted']]
rf8FitPD['Predicted'] = rankdata(rf8FitPD.Predicted,method='min')

In [16]:
rfClass8 = pd.merge(season8,rf8FitPD).sort_values('Predicted')
rfClass8

,Name,Actual,Predicted
0,Acid Betty,0,1
6,Kim Chi,0,2
10,Robbie Turner,0,3
1,Bob the Drag Queen,0,4
2,Chi Chi DeVayne,0,4
5,Derrick Barry,0,6
11,Thorgy Thor,0,7
8,Naomi Smalls,0,8
9,Naysha Lopez,9,8
4,Dax ExclamationPoint,11,10


# Random Forest Regressor

In [17]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(200,random_state=24601)
forest.fit(Xtrain,ytrain)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=False, random_state=24601,
           verbose=0, warm_start=False)

## Season 7 Predictions

In [18]:
forestFit = forest.predict(Xtest)
forestFitDB = pd.DataFrame(forestFit.round())
forestFitDB['Name'] = season7.Name
forestFitDB.columns = ['Predicted','Name']
forestFitDB['Predicted'] = rankdata(forestFitDB.Predicted,method='min')
rfReg7 = pd.merge(season7,forestFitDB).sort_values('Actual')
rfReg7

,Name,Actual,Predicted
13,Voilet Chachki,1,2
0,Ginger Minj,2,1
9,Pearl,3,6
5,Kennedy Davenport,4,2
4,Katya,5,2
12,Trixie Mattel,6,8
7,Miss Fame,7,8
1,Jaidynn Diore Fierce,8,6
6,Max,9,5
3,Kandy Ho,10,8


In [19]:
kendalltau(rfReg7.Actual,rfReg7.Predicted)

KendalltauResult(correlation=0.73379938570534309, pvalue=0.00025655048033250812)

## Season 8 Predictions

In [20]:
forest8 = forest.predict(season8X)
forest8DB = pd.DataFrame(forest8.round())
forest8DB['Name'] = season8.Name
forest8DB.columns = ['Predicted','Name']
forest8DB['Predicted'] = rankdata(forest8DB.Predicted,method='min')
pd.merge(season8,forest8DB).sort_values('Predicted')

,Name,Actual,Predicted
6,Kim Chi,0,1
1,Bob the Drag Queen,0,2
2,Chi Chi DeVayne,0,2
10,Robbie Turner,0,2
0,Acid Betty,0,5
5,Derrick Barry,0,5
8,Naomi Smalls,0,7
11,Thorgy Thor,0,7
7,Laila McQueen,11,9
4,Dax ExclamationPoint,11,10


# Neural Network

Unfortunately, scikit learn's stable release does not include neural networks (it is being added in the next version) so if I want to include a neural network I have to code one from scratch (credit goes to XXX)

In [12]:
import numpy as np
import math, random
# create the "step" function
def sigmoid(t): 
    return 1 / (1 + math.exp(-t))

# create the neuron
def neuron_output(weights, inputs):   #This is a simpler representation; weights for input plus one extra (bias)
    return sigmoid(np.dot(weights, inputs))

# define the network
def feed_forward(neural_network, input_vector):
    """takes in a neural network (represented as a list of lists of lists of weights)
    and returns the output from forward-propagating the input"""

    outputs = []

    for layer in neural_network: #Remember the neural network is given as a list of "layers" which have neurons in them

        input_with_bias = input_vector + [1]          # add a bias input (this just allos us to use a dot product)
        output = [neuron_output(neuron, input_with_bias) # compute the output
                  for neuron in layer]                   # for this layer
        outputs.append(output)                           # and remember it

        # the input to the next layer is the output of this one
        input_vector = output

    return outputs

# define the back-propagation that allows the network to learn
def backpropagate(network, input_vector, target):

    hidden_outputs, outputs = feed_forward(network, input_vector)
    
    # the output * (1 - output) is from the derivative of sigmoid
    output_deltas = [output * (1 - output) * (output - target[i])
                     for i, output in enumerate(outputs)]
                     
    # adjust weights for output layer (network[-1])
    for i, output_neuron in enumerate(network[-1]):
        for j, hidden_output in enumerate(hidden_outputs + [1]):
            output_neuron[j] -= output_deltas[i] * hidden_output

    # back-propagate errors to hidden layer
    hidden_deltas = [hidden_output * (1 - hidden_output) * 
                      np.dot(output_deltas, [n[i] for n in network[-1]]) 
                     for i, hidden_output in enumerate(hidden_outputs)]

    # adjust weights for hidden layer (network[0])
    for i, hidden_neuron in enumerate(network[0]):
        for j, input in enumerate(input_vector + [1]):
            hidden_neuron[j] -= hidden_deltas[i] * input

In [16]:
random.seed(0)   # to get repeatable results
input_size = 8  # each input is a vector of length 8
num_hidden = 5   # we'll have 5 neurons in the hidden layer
output_size = 14 # we need 14 outputs for each input, since there are 14 possible places

In [17]:
# each hidden neuron has one weight per input, plus a bias weight
hidden_layer = [[random.random() for __ in range(input_size + 1)]
                    for __ in range(num_hidden)]

In [18]:
# each output neuron has one weight per hidden neuron, plus a bias weight
output_layer = [[random.random() for __ in range(num_hidden + 1)]
                    for __ in range(output_size)]

In [20]:
# the network starts out with random weights
network = [hidden_layer, output_layer]

Neural networks work best if values are standardized close to 0-1. So for the non-dummy variables, I'll standardize to mean=0, sd=1

In [34]:
from sklearn.preprocessing import scale
queensNN = queens
queensNN['Age'] = scale(queensNN['Age'])
queensNN['Wins'] = scale(queensNN['Wins'])
queensNN['Highs'] = scale(queensNN['Highs'])
queensNN['Lows'] = scale(queensNN['Lows'])
queensNN['Season'] = scale(queensNN['Season'])
queensNN['Lipsyncs'] = scale(queensNN['Lipsyncs'])

#dataset = zip(inputs_dig, targets_dig)

In [98]:
XtrainNN = queensNN.loc[queens.Season<7,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
XtestNN = queensNN.loc[(queensNN.Season>1) & (queensNN.Season<1.5),['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
ytrainNN = queensNN.loc[queens.Season<7,'Place']
ytestNN = queensNN.loc[queens.Season==7,'Place']
season8XNN = queensNN.loc[queensNN.Season>1.5,['Age','PuertoRico','PlusSize','Wins','Highs','Lows','Lipsyncs','Season']]
season8yNN = queensNN.loc[queens.Season==8,'Place']

In [43]:
yNN = [[1 if i == j else 0 for i in range(14)]
               for j in ytrainNN ]
XNN = XtrainNN.values.tolist()

In [44]:
# 10,000 iterations seems enough to converge
for i in range(10000):
    for input_vector, target_vector in zip(XNN, yNN):
        backpropagate(network, input_vector, target_vector)

In [45]:
def predict(input):
        return feed_forward(network, input)[-1]

## Season 7 Predictions

In [70]:
X7NN = XtestNN.values.tolist()

In [92]:
predictedY = []
for i, input in enumerate(X7NN):
    outputs = predict(input)
    predictedY.append(outputs)
#for q in predictedY:
#    print([round(a,2) for a in q])

In [93]:
predictedPlace = [a.index(max(a)) for a in predictedY]
predictedPlace = rankdata(predictedPlace,method='min')
season7NN = season7
season7NN['Predicted'] = predictedPlace
season7NN

,Name,Actual,Predicted
0,Ginger Minj,2,2
1,Jaidynn Diore Fierce,8,6
2,Jasmine Masters,12,13
3,Kandy Ho,10,10
4,Katya,5,5
5,Kennedy Davenport,4,4
6,Max,9,9
7,Miss Fame,7,7
8,Mrs. Kasha Davis,11,11
9,Pearl,3,3


In [94]:
kendalltau(season7NN.Actual,season7NN.Predicted)

KendalltauResult(correlation=0.88399139217572797, pvalue=1.0634406379807569e-05)

## Season 8 Predictions

In [106]:
season8XNN['Wins'] = scale(season8XNN['Wins'])
season8XNN['Highs'] = scale(season8XNN['Highs'])
season8XNN['Lows'] = scale(season8XNN['Lows'])
season8XNN['Lipsyncs'] = scale(season8XNN['Lipsyncs'])
X8NN = season8XNN.values.tolist()
predictedY = []
for i, input in enumerate(X8NN):
    outputs = predict(input)
    predictedY.append(outputs)
for q in predictedY:
    print([round(a,2) for a in q])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.54, 0.27, 0.1]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02, 1.0, 0.02, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.08, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13, 0.99, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.54, 0.28, 0.11]
[0.99, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.0, 0.0, 0.01]
[0.0, 0.84, 0.06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53, 0.26, 0.09]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03, 0.0, 0.0, 0.94, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04, 0.96, 0.01, 0.15, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.96, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.54, 0.28, 0.11]


In [107]:
[a.index(max(a)) for a in predictedY]

[11, 9, 9, 10, 11, 0, 1, 11, 12, 9, 10, 11]

In [108]:
predictedPlace = [a.index(max(a)) for a in predictedY]
predictedPlace = rankdata(predictedPlace,method='min')
season8NN = season8
season8NN['Predicted'] = predictedPlace
season8NN

,Name,Actual,Predicted
0,Acid Betty,0,8
1,Bob the Drag Queen,0,3
2,Chi Chi DeVayne,0,3
3,Cynthia Lee Fontaine,10,6
4,Dax ExclamationPoint,11,8
5,Derrick Barry,0,1
6,Kim Chi,0,2
7,Laila McQueen,11,8
8,Naomi Smalls,0,12
9,Naysha Lopez,9,3
